In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
import psycopg2

In [2]:
# create filepaths
dow_jones_path = os.path.join('Resources', 'DowJones.csv')
nasdaq_path = os.path.join('Resources', 'Nasdaq.csv')
sp500_path = os.path.join('Resources', 'SP500.csv')

In [3]:
# read in files
dow_jones_df = pd.read_csv(dow_jones_path)
nasdaq_df = pd.read_csv(nasdaq_path)
sp500_df = pd.read_csv(sp500_path)

In [4]:
# added to prevent merge error later on
dow_jones_df.Date = pd.to_datetime(dow_jones_df.Date)
nasdaq_df.Date = pd.to_datetime(nasdaq_df.Date)
sp500_df.Date = pd.to_datetime(sp500_df.Date)

In [5]:
# rename columns
dow_jones_df = dow_jones_df.rename(columns = {'Value': 'Dow_Jones'})
nasdaq_df = nasdaq_df.rename(columns = {'Value': 'NASDAQ'})
sp500_df = sp500_df.rename(columns = {'Value': 'S&P_500'})

In [6]:
# connect to database
from config import username, pwd
from db_config import endpoint, port, db_name
db_url = f'postgresql://{username}:{pwd}@{endpoint}:{port}/{db_name}'
engine = create_engine(db_url)
conn = engine.connect()

In [7]:
# upload to database
dow_jones_df.to_sql(name = 'dow_jones', con = engine, if_exists = 'replace', index = False)
nasdaq_df.to_sql(name = 'nasdaq', con = engine, if_exists = 'replace', index = False)
sp500_df.to_sql(name = 'sp_500', con = engine, if_exists = 'replace', index = False)

In [8]:
# SQL join query
query = '''select coalesce(s."Date", n."Date", d."Date") as "Date", "S&P_500", "NASDAQ", "Dow_Jones"
from sp_500 s
full outer join nasdaq n
on s."Date" = n."Date"
full outer join dow_jones d
on s."Date" = d."Date"'''

# create dataframe by reading from database
combined_df = pd.read_sql(query, con = conn)

# write to database
combined_df.to_sql('joined_stocks', con = engine, if_exists = 'replace', index = False)

# close connection
conn.close()

In [9]:
# export locally
path = os.path.join('Resources', 'joined_stocks.csv')
combined_df.to_csv(path, index = False)